In [1]:
import os 
os.chdir("../../../../")
import random
import numpy
import numpy as np
import pandas as pd
import datetime

from scripts.python.PdfParse import *
from scripts.python.utils import *
from scripts.python.ts_utils import *

from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis

In [2]:
tonga_folder = os.getcwd() + "/data/tourism/tonga/temp/"
tonga_files = os.listdir(tonga_folder)
filepaths = [tonga_folder + file for file in tonga_files]

## Tonga Official Statistics
### 2019 and Later

In [3]:
monthly_temp_file_path = os.getcwd() + "/data/tourism/tonga/tonga_monthly_visitor.csv"
tonga = (pd.read_csv(monthly_temp_file_path)
         .drop("Unnamed: 0", axis=1))

# Subset the df by only including dates later than 2019
tonga_monthly_aft19 = (tonga[tonga.Year >= 2019]
                       .drop_duplicates()
                       .reset_index()
                       .drop(["index"], axis=1)
                       .sort_values(by="date", ascending=True)
                       .fillna(0))

tonga_monthly_aft19.head(5)

Year   Air    Ship  Yacht  Total        date
0  2019  4372   313.0    3.0   4688  2019-01-01
1  2019  2709  3448.0    0.0   6157  2019-02-01
2  2019  3109  5570.0    7.0   8686  2019-03-01
3  2019  3574  3508.0   13.0   7095  2019-04-01
4  2019  4183  3508.0   13.0   7704  2019-04-01

In [4]:
# Check whether Total = the sum of (Air, Yacht, Ship)
error_dict = {}
for idx in tonga_monthly_aft19.index:
    cols = ["Air", "Ship", "Yacht"]
    row_sum = 0
    for col in cols:
        row_sum += tonga_monthly_aft19[col][idx]
    if row_sum == tonga_monthly_aft19["Total"][idx]:
        pass
    else:
        error_dict.update({tonga_monthly_aft19["date"][idx]: idx})
        
print(error_dict)

{'2020-09-01': 24, '2020-10-01': 27, '2020-11-01': 28, '2020-12-01': 31, '2021-01-01': 32, '2021-02-01': 33, '2021-03-01': 34, '2021-05-01': 36, '2021-06-01': 37, '2021-07-01': 38, '2021-08-01': 39}


In [5]:
# Produce the duplicated entries
dupl_lst = tonga_monthly_aft19[tonga_monthly_aft19["date"].duplicated()
                               == True]["date"].to_list()

for date in dupl_lst:
    if date in error_dict.keys():
        tonga_monthly = tonga_monthly_aft19.drop(error_dict[date])
        error_dict.pop(date)
        dupl_lst.remove(date)

In [6]:
tonga_monthly_max = (tonga_monthly_aft19.drop_duplicates()
                     .reset_index()
                     .drop("index", axis=1))

tonga_monthly_min = tonga_monthly_max.copy()


for date in dupl_lst:
    dupl_total = tonga_monthly_max[tonga_monthly_max.date == date]["Total"]
    if len(dupl_total) > 1:
        min_num, max_num = dupl_total.min(), dupl_total.max()
        min_idx = tonga_monthly_max[tonga_monthly_max.Total == min_num].index
        max_idx = tonga_monthly_min[tonga_monthly_min.Total == max_num].index
        tonga_monthly_max = tonga_monthly_max.drop(min_idx[0], axis=0)
        tonga_monthly_min = tonga_monthly_min.drop(max_idx[0], axis=0)


# Change the datetime format for tonga_monthly_min, tonga_monthly_max
tonga_monthly_max = tonga_monthly_max.reset_index().drop("index", axis=1)
tonga_monthly_min = tonga_monthly_min.reset_index().drop("index", axis=1)


tonga_monthly_max["date"], tonga_monthly_min["date"] = \
    pd.to_datetime(tonga_monthly_max["date"]), pd.to_datetime(
        tonga_monthly_min["date"])


tonga_monthly_max["Year"], tonga_monthly_max["Month"] = \
    tonga_monthly_max["date"].dt.year, tonga_monthly_max["date"].dt.month

tonga_monthly_min["Year"], tonga_monthly_min["Month"] = \
    tonga_monthly_min["date"].dt.year, tonga_monthly_min["date"].dt.month

In [7]:
tonga_monthly_max.head(30)

Year   Air    Ship  Yacht  Total       date  Month
0   2019  4372   313.0    3.0   4688 2019-01-01      1
1   2019  2709  3448.0    0.0   6157 2019-02-01      2
2   2019  3109  5570.0    7.0   8686 2019-03-01      3
3   2019  4183  3508.0   13.0   7704 2019-04-01      4
4   2019  5166  1506.0  128.0   6800 2019-05-01      5
5   2019  6647  1798.0  489.0   8934 2019-06-01      6
6   2019  8121     0.0  194.0   8315 2019-07-01      7
7   2019  7890     0.0  288.0   8178 2019-08-01      8
8   2019  6811   636.0  382.0   7829 2019-09-01      9
9   2019  5649  2178.0  428.0   8255 2019-10-01     10
10  2019  4938  2712.0   72.0   7722 2019-11-01     11
11  2019  7689  2998.0   17.0  10704 2019-12-01     12
12  2020  4665   852.0    0.0   5517 2020-01-01      1
13  2020  3008  1678.0    6.0   4692 2020-02-01      2
14  2020  1259     0.0    0.0   1259 2020-03-01      3
15  2020     0     0.0    0.0      0 2020-04-01      4
16  2020     0     0.0    0.0      0 2020-05-01      5
17  2020     0     0.0    0.0      0 2020-06-01      6
18  2020     1     0.0    0.0      1 2020-07-01      7
19  2020     5     0.0    0.0      5 2020-08-01      8
20  2020     1     6.0    0.0     16 2020-09-01      9
21  2020    16     0.0    0.0     16 2020-09-01      9
22  2020     1     3.0    0.0     13 2020-10-01     10
23  2020     2     0.0    0.0     20 2020-11-01     11
24  2020    20     0.0    0.0     20 2020-11-01     11
25  2020     2     3.0    0.0     23 2020-12-01     12
26  2021     1     8.0    0.0     18 2021-01-01      1
27  2021     4     1.0    0.0     41 2021-02-01      2
28  2021     2     3.0    0.0     23 2021-03-01      3
29  2021     2     0.0    0.0      2 2021-04-01      4

### Before 2019

In [8]:
tonga = (pd.read_csv(monthly_temp_file_path)
         .drop("Unnamed: 0", axis=1))

# subset to the pre-19
tonga_monthly_pre19 = (tonga[tonga.Year < 2019]
                       .drop_duplicates()
                       .sort_values(by="date", ascending=True)
                       .reset_index()
                       .drop(["index"], axis=1))

print(check_quality(tonga_monthly_pre19, ["date", "Year"], "Total"))

[97]


In [9]:
tonga_monthly_pre19 = (tonga_monthly_pre19.drop(index=[60, 97], axis=1)
                       .drop_duplicates()
                       .reset_index()
                       .drop("index", axis=1))
tonga_monthly_pre19.groupby(by="Year").count()

Air  Ship  Yacht  Total  date
Year                               
2010   12    12     12     12    12
2011   12    11     12     12    12
2012   12    12     12     12    12
2013   12    12     12     12    12
2014   12    12     12     12    12
2015   12    12     12     12    12
2016   12    12     12     12    12
2017   12    12     12     12    12
2018   12    12     12     12    12

In [13]:
tonga = (pd.concat([tonga_monthly_pre19, tonga_monthly_max.drop("Month", axis=1)], axis=0)
           .rename({"time": "date"}, axis=1))
tonga.to_csv(os.getcwd() +
             "/data/tourism/tonga/intermediate/tonga_monthly_visitor.csv",
             encoding="utf-8")

In [14]:
get_adf_df(tonga, ["Total"])

Test Statistic   p-value  # Lags Used  Number of Observations Used  \
Total       -1.147026  0.695959         10.0                        135.0   

       Critical Value (1%)  Critical Value (5%)  Critical Value (10%)  
Total            -3.479743            -2.883198              -2.57832

## Aviation Data
### Correlation Analysis
#### Using Maximum for duplicated rows

In [12]:
aviation_path = os.getcwd() + "/data/tourism/aviation_seats_flights_pic.xlsx"
aviation = pd.read_excel(aviation_path)
aviation.head(5)

Country ISO               Region       Date Aircraft_type  \
0    Fiji  FJ  East Asia & Pacific 2019-01-01     passenger   
1    Fiji  FJ  East Asia & Pacific 2019-01-02     passenger   
2    Fiji  FJ  East Asia & Pacific 2019-01-03     passenger   
3    Fiji  FJ  East Asia & Pacific 2019-01-04     passenger   
4    Fiji  FJ  East Asia & Pacific 2019-01-05     passenger   

   Seats_arrivals_domestic  Seats_arrivals_interregional  \
0                      839                           273   
1                      974                           313   
2                     1190                           443   
3                      831                           586   
4                      744                           273   

   Seats_arrivals_intraregional  Seats_arrivals_intl  Seats_arrivals_total  \
0                          3480                 3753                  4592   
1                          3471                 3784                  4758   
2                          3675                 4118                  5308   
3                          3159                 3745                  4576   
4                          4752                 5025                  5769   

   Available_seat_kilometers  Number_of_flights_domestic  \
0               1.430416e+07                           8   
1               1.495610e+07                           8   
2               1.592143e+07                          10   
3               1.457334e+07                           7   
4               1.773449e+07                           7   

   Number_of_flights_interregional  Number_of_flights_intraregional  \
0                                1                               10   
1                                1                               10   
2                                2                               12   
3                                2                               12   
4                                1                               12   

   Number_of_flights_intl  Number_of_flights_total  
0                      11                       19  
1                      11                       19  
2                      14                       24  
3                      14                       21  
4                      13                       20

In [17]:
tg_avi = (aviation[(aviation.ISO == "TO") & (aviation.Aircraft_type == "passenger")]
          .reset_index()
          .drop("index", axis=1)
          [["Date", "Number_of_flights_intl", "Number_of_flights_total",
            "Seats_arrivals_intl", "Seats_arrivals_total"]])

dates = pd.DataFrame(pd.date_range(start="2019-01-01",
                                   end="2021-12-31"), columns=["Date"])

tg_avi = dates.merge(tg_avi, how="left", on="Date")
tg_avi["Date"] = pd.to_datetime(tg_avi["Date"])
tg_avi["Month"], tg_avi["Year"] = tg_avi["Date"].dt.month, tg_avi["Date"].dt.year

tg_avi_19_21 = tg_avi.groupby(by=["Year", "Month"]).sum().reset_index()

In [23]:
tg_max_merged = tg_avi_19_21.merge(
    tonga_monthly_max, on=["Year", "Month"], how="left")
tg_max_merged.to_csv(os.getcwd() + "/data/tourism/tonga/intermediate/tg_merged.csv", encoding="utf-8")
tg_max_merged.head(5)

Year  Month  Number_of_flights_intl  Number_of_flights_total  \
0  2019      1                    68.0                    102.0   
1  2019      2                    45.0                     80.0   
2  2019      3                    55.0                    107.0   
3  2019      4                    53.0                     78.0   
4  2019      5                    64.0                     87.0   

   Seats_arrivals_intl  Seats_arrivals_total   Air    Ship  Yacht  Total  \
0              14706.0               17698.0  4372   313.0    3.0   4688   
1               9740.0               11440.0  2709  3448.0    0.0   6157   
2              10236.0               12662.0  3109  5570.0    7.0   8686   
3              10615.0               10919.0  4183  3508.0   13.0   7704   
4              11858.0               12226.0  5166  1506.0  128.0   6800   

        date  
0 2019-01-01  
1 2019-02-01  
2 2019-03-01  
3 2019-04-01  
4 2019-05-01

In [22]:
from scipy.stats import pearsonr


corr_seats_max, _ = pearsonr(tg_max_merged["Seats_arrivals_total"], tg_max_merged["Total"])
corr_seat_flight_max, _ = pearsonr(tg_max_merged["Number_of_flights_total"], tg_max_merged["Total"])
print(" Using the Maximum Values of the duplicated rows:  \n",
f"Pearson Correlation between FlightRadar's Seats Arrival and TG's census data is{corr_seats_max: .4f}. \n",
f"Pearson Correlation between FlightRadar's # of Flights Arrival and TG's census data is{corr_seat_flight_max: .4f}.")

# Correlation Matrix
cols = ["Seats_arrivals_intl", "Seats_arrivals_total",
        "Number_of_flights_total", "Number_of_flights_intl", "Total"]
corr_matrix_max = tg_max_merged[cols].corr()
corr_matrix_max

 Using the Maximum Values of the duplicated rows:  
 Pearson Correlation between FlightRadar's Seats Arrival and TG's census data is 0.9416. 
 Pearson Correlation between FlightRadar's # of Flights Arrival and TG's census data is 0.9536.


Seats_arrivals_intl  Seats_arrivals_total  \
Seats_arrivals_intl                 1.000000              0.995625   
Seats_arrivals_total                0.995625              1.000000   
Number_of_flights_total             0.962869              0.975549   
Number_of_flights_intl              0.987192              0.982143   
Total                               0.947912              0.941639   

                         Number_of_flights_total  Number_of_flights_intl  \
Seats_arrivals_intl                     0.962869                0.987192   
Seats_arrivals_total                    0.975549                0.982143   
Number_of_flights_total                 1.000000                0.972483   
Number_of_flights_intl                  0.972483                1.000000   
Total                                   0.953601                0.957868   

                            Total  
Seats_arrivals_intl      0.947912  
Seats_arrivals_total     0.941639  
Number_of_flights_total  0.953601  
Number_of_flights_intl   0.957868  
Total                    1.000000

In [11]:
tg_min_merged = tg_avi_19_21.merge(
    tonga_monthly_min, on=["Year", "Month"], how="left").drop("time", axis=1)



corr_seats_min, _ = pearsonr(tg_min_merged["Seats_arrivals_total"], tg_min_merged["Total"])
corr_seat_flight_min, _ = pearsonr(tg_min_merged["Number_of_flights_total"], tg_min_merged["Total"])
print(" Using the Maximum Values of the duplicated rows:  \n",
f"Pearson Correlation between FlightRadar's Seats Arrival and TG's census data is{corr_seats_min: .4f}. \n",
f"Pearson Correlation between FlightRadar's # of Flights Arrival and TG's census data is{corr_seat_flight_min: .4f}.")

# Correlation Matrix
corr_matrix_min = tg_min_merged[cols].corr()
corr_matrix_min

 Using the Maximum Values of the duplicated rows:  
 Pearson Correlation between FlightRadar's Seats Arrival and TG's census data is 0.9401. 
 Pearson Correlation between FlightRadar's # of Flights Arrival and TG's census data is 0.9489.


Seats_arrivals_intl  Seats_arrivals_total  \
Seats_arrivals_intl                 1.000000              0.995476   
Seats_arrivals_total                0.995476              1.000000   
Number_of_flights_total             0.961636              0.974741   
Number_of_flights_intl              0.986774              0.981558   
Total                               0.944631              0.940102   

                         Number_of_flights_total  Number_of_flights_intl  \
Seats_arrivals_intl                     0.961636                0.986774   
Seats_arrivals_total                    0.974741                0.981558   
Number_of_flights_total                 1.000000                0.971579   
Number_of_flights_intl                  0.971579                1.000000   
Total                                   0.948930                0.948709   

                            Total  
Seats_arrivals_intl      0.944631  
Seats_arrivals_total     0.940102  
Number_of_flights_total  0.948930  
Number_of_flights_intl   0.948709  
Total                    1.000000

### Cross-correlation

In [12]:
tg_cc = cross_corr_df(tg_max_merged, "Total", "Seats_arrivals_intl")
tg_cc.head(5)

lag  cross_corr_coef
0    0         0.946237
1    1         0.875779
2    2         0.813119
3    3         0.695951
4    4         0.602881

### Stationarity
#### ADF test
$H_{0} $: The Series is non-stationary.

In [13]:
incl_cols = tg_max_merged.columns[2:].to_list()
adf_df = get_adf_df(tg_max_merged, incl_cols)
adf_df

Test Statistic   p-value  # Lags Used  \
Number_of_flights_intl        -0.646765  0.860061          1.0   
Number_of_flights_total       -2.115997  0.238131          5.0   
Seats_arrivals_intl           -1.018914  0.746303          5.0   
Seats_arrivals_total          -1.082484  0.722006          5.0   
Total                         -3.332505  0.013487         10.0   

                         Number of Observations Used  Critical Value (1%)  \
Number_of_flights_intl                          34.0            -3.639224   
Number_of_flights_total                         30.0            -3.669920   
Seats_arrivals_intl                             30.0            -3.669920   
Seats_arrivals_total                            30.0            -3.669920   
Total                                           25.0            -3.723863   

                         Critical Value (5%)  Critical Value (10%)  
Number_of_flights_intl             -2.951230             -2.614447  
Number_of_flights_total            -2.964071             -2.621171  
Seats_arrivals_intl                -2.964071             -2.621171  
Seats_arrivals_total               -2.964071             -2.621171  
Total                              -2.986489             -2.632800

__`Total` is stationary while none of other indicators from GAD is stationary.__

### KPSS test
$H_{0} $: The Series is stationary.

In [14]:
kpss_test(tg_max_merged, incl_cols)

Test statistic  p-value  Critical value - 1%  \
Number_of_flights_intl           0.1377   0.0654                0.216   
Number_of_flights_total          0.1722   0.0281                0.216   
Seats_arrivals_intl              0.1216   0.0952                0.216   
Seats_arrivals_total             0.1312   0.0774                0.216   
Total                            0.1349   0.0705                0.216   

                         Critical value - 5%  Critical value - 10%  
Number_of_flights_intl                 0.146                 0.119  
Number_of_flights_total                0.146                 0.119  
Seats_arrivals_intl                    0.146                 0.119  
Seats_arrivals_total                   0.146                 0.119  
Total                                  0.146                 0.119

#### Order of Integration for Stationarity

In [15]:
tg_log = pd.DataFrame()
for col in tg_max_merged.columns[2:]:
    tg_log[str(col)+"_log"] = np.log(tg_max_merged[col]+1)

# first- and second-order differencing    
tg_diff1 = tg_max_merged[incl_cols].diff().dropna()
tg_diff2 = tg_diff1.diff().dropna()

In [16]:
get_adf_df(tg_diff1, incl_cols)

Test Statistic   p-value  # Lags Used  \
Number_of_flights_intl        -5.643475  0.000001          0.0   
Number_of_flights_total       -2.209763  0.202721         10.0   
Seats_arrivals_intl           -2.170531  0.217114          5.0   
Seats_arrivals_total          -1.967797  0.300869          5.0   
Total                         -2.878906  0.047856         10.0   

                         Number of Observations Used  Critical Value (1%)  \
Number_of_flights_intl                          34.0            -3.639224   
Number_of_flights_total                         24.0            -3.737709   
Seats_arrivals_intl                             29.0            -3.679060   
Seats_arrivals_total                            29.0            -3.679060   
Total                                           24.0            -3.737709   

                         Critical Value (5%)  Critical Value (10%)  
Number_of_flights_intl             -2.951230             -2.614447  
Number_of_flights_total            -2.992216             -2.635747  
Seats_arrivals_intl                -2.967882             -2.623158  
Seats_arrivals_total               -2.967882             -2.623158  
Total                              -2.992216             -2.635747

In [17]:
adf_df_diff2 = get_adf_df(tg_diff2, incl_cols)
adf_df_diff2

Test Statistic   p-value  # Lags Used  \
Number_of_flights_intl        -2.468431  0.123345         10.0   
Number_of_flights_total       -2.573241  0.098655         10.0   
Seats_arrivals_intl           -4.410197  0.000285          3.0   
Seats_arrivals_total          -4.545829  0.000162          3.0   
Total                         -2.291150  0.174838          8.0   

                         Number of Observations Used  Critical Value (1%)  \
Number_of_flights_intl                          23.0            -3.752928   
Number_of_flights_total                         23.0            -3.752928   
Seats_arrivals_intl                             30.0            -3.669920   
Seats_arrivals_total                            30.0            -3.669920   
Total                                           25.0            -3.723863   

                         Critical Value (5%)  Critical Value (10%)  
Number_of_flights_intl             -2.998500             -2.638967  
Number_of_flights_total            -2.998500             -2.638967  
Seats_arrivals_intl                -2.964071             -2.621171  
Seats_arrivals_total               -2.964071             -2.621171  
Total                              -2.986489             -2.632800

__`Seats_arrivals_intl` and `Seats_arrivals_total` are stationary at second-order differencing.__

###  Grangers Causality

In [22]:
grangers_causation_matrix(tg_diff2,
                          variables=["Total", "Seats_arrivals_intl"],
                          maxlag=10)

Total_x  Seats_arrivals_intl_x
Total_y                    1.0                    0.0
Seats_arrivals_intl_y      0.0                    1.0

In [27]:
from statsmodels.tsa.api import VAR

tg_train = tg_max_merged[["Total", "Seats_arrivals_intl"]]

model = VAR(tg_train)
res = model.select_order(10)
res.summary()

<class 'statsmodels.iolib.table.SimpleTable'>

In [28]:
model_fit = model.fit(maxlags=9)
model_fit.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 09, Jan, 2023
Time:                     16:42:05
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    27.4201
Nobs:                     27.0000    HQIC:                   26.1386
Log likelihood:          -384.173    FPE:                2.58953e+11
AIC:                      25.5963    Det(Omega_mle):     8.92138e+10
--------------------------------------------------------------------
Results for equation Total
                            coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------------------
const                       3033.013874      4151.878442            0.731           0.465
L1.Total                       1.179010         0.545001            2.163           0.031
L1.Seats_arrivals_intl  

In [29]:
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fit.resid)

for col, val in zip(tg_train.columns, out):
    print(f"{col}", ':', round(val, 2))

Total : 1.6
Seats_arrivals_intl : 1.96
